The folder flappy_bird_gym comes from https://github.com/Talendar/flappy-bird-gym/tree/main/flappy_bird_gym. It provides a standard implementation of the flappy bird game and allows us to pull metrics of the game. We slightly modified it to experiment with different state representations, reward structures, and fixed a few bugs in the recorder.

In [ ]:
!pip install pygame
!pip install keras-rl2==1.0.4
!pip install tensorflow==2.11
!pip install moviepy==1.0.3

fatal: destination path 'flappy-bird-gym' already exists and is not an empty directory.


In [82]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import flappy_bird_gym
from flappy_bird_gym.recorder import Recorder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

env = flappy_bird_gym.make("FlappyBird-v0")

  deprecation(

  deprecation(



**Building The Neural Network**

In [83]:
def build_model(input, actions):
    model = Sequential()

    model.add(Dense(64, activation='relu', input_shape=(1, input)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(225, activation='relu'))
    model.add(Dense(128, activation='relu'))

    model.add(Dense(actions, activation='linear'))
    model.summary()
    return model

In [84]:
obs = env.observation_space.shape[0]
actions = env.action_space.n
model = build_model(obs, actions)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_85 (Dense)            (None, 1, 64)             192       
                                                                 
 dense_86 (Dense)            (None, 1, 128)            8320      
                                                                 
 dense_87 (Dense)            (None, 1, 225)            29025     
                                                                 
 dense_88 (Dense)            (None, 1, 128)            28928     
                                                                 
 dense_89 (Dense)            (None, 1, 2)              258       
                                                                 
Total params: 66,723
Trainable params: 66,723
Non-trainable params: 0
_________________________________________________________________


In [86]:
del model
model = build_model(obs, actions)

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_90 (Dense)            (None, 1, 64)             192       
                                                                 
 dense_91 (Dense)            (None, 1, 128)            8320      
                                                                 
 dense_92 (Dense)            (None, 1, 225)            29025     
                                                                 
 dense_93 (Dense)            (None, 1, 128)            28928     
                                                                 
 dense_94 (Dense)            (None, 1, 2)              258       
                                                                 
Total params: 66,723
Trainable params: 66,723
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=0.5, value_min=0.0001, value_test=0.0, nb_steps=3000000)
    memory = SequentialMemory(limit=100000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   enable_dueling_network=True, dueling_type='avg',
                   nb_actions=actions, nb_steps_warmup=1000)
    return dqn

dqn = build_agent(model, actions)

In [ ]:
dqn.compile(Adam(learning_rate=0.00025))
dqn.fit(env, nb_steps=5000000, visualize=False, verbose=0)

In [ ]:
env = flappy_bird_gym.make("FlappyBird-v0")
directory = './video'
env = Recorder(env, directory)

dqn.test(env, visualize=False, nb_episodes=1)

Testing for 1 episodes...
Episode 1: reward: 5140.000, steps: 5140

